# 前言
这个项目是基于ppdet，做钢铁缺陷检测，因为刚报名没有多久，因此先训练了picodet-L和ppyoloe-plus-s。最开始的目的只是像比对一下两个模型，然后再基于前面做的实验修改，想办法提点。当然可能本人能力有限，有错误的地方希望各位大佬留言批评。

# 一.安装PaddleDetection
PaddleDetection为基于飞桨PaddlePaddle的端到端目标检测套件，内置30+模型算法及300+预训练模型，覆盖目标检测、实例分割、跟踪、关键点检测等方向，其中包括服务器端和移动端高精度、轻量级产业级SOTA模型、冠军方案和学术前沿算法，并提供配置化的网络模块组件、十余种数据增强策略和损失函数等高阶优化支持和多种部署方案，在打通数据处理、模型开发、训练、压缩、部署全流程的基础上，提供丰富的案例及教程，加速算法产业落地应用。


ppdet，用一时，爽一时，一直用，一直爽！

先进入work目录，因为work目录关闭了是可以保存下来的
```
cd /home/aistudio/work
```

我们然后把ppdetclone下来：
```
git clone https://github.com/PaddlePaddle/PaddleDetection.git
```

安装依赖
```
cd PaddleDetection
pip install -r requirements.txt
```

编译安装
```
python setup.py install
```

# 二.构建数据集管道
在创建Ai stduio项目的时候就把选择钢铁缺陷检测的数据集，这个时候data目录下面就会有钢铁缺陷检测的数据集了，为了方便，我再work目录下创建了data目录，把数据集移动到了这里。
```
mkdir /home/aistudio/work/data
mv /home/aistudio/data/data169999/*  /home/aistudio/work/data
```

然后就是解压
```
cd /home/aistudio/work/data
unzip train.zip
unzip test.zip
```

随后按照ppdet voc格式数据集读取的要求需要写入一个txt文件，这里我有两个选择，一个是将train数据全部用来训练，训练过程不评估，我一开始是这么做的。然后发现学习资料里面的项目分出来了一部分数据计算map，所以我也加了这一部分，毕竟担心过拟合问题，但是这样的话，毕竟训练数据会缺失，因此我不是特别推荐这样的做法。

代码如下：
```
将路径信息写入txt
def txt_get_all(img_dir,xml_dir,txt_path):
    xml_list=glob.glob(os.path.join(xml_dir,'*.xml'))
    with open(txt_path,'w') as f:
        for xml in tqdm(xml_list):
            file_name=os.path.basename(xml)
            file_name = file_name.split('.')[0]
            img_path = os.path.join(img_dir,file_name + '.jpg')
            if not os.path.exists(img_path):
                print(img_path + ' not exists')
                continue
            str=img_path+' '+xml
            f.write(str)
            f.write('\n')
```

```
def txt_get_spilt(img_dir,xml_dir,train_txt,test_txt):
    xml_list=glob.glob(os.path.join(xml_dir,'*.xml'))
    xml_num=len(xml_list)
    test_num=int(xml_num/14)
    test_id=np.random.choice(xml_num,test_num,replace=False).tolist()
    train_f=open(train_txt,'w')
    test_f=open(test_txt,'w')
    for i,xml in enumerate(tqdm(xml_list)):
        file_name=os.path.basename(xml)
        file_name = file_name.split('.')[0]
        img_path = os.path.join(img_dir,file_name + '.jpg')
        if not os.path.exists(img_path):
            print(img_path + ' not exists')
            continue
        str=img_path+' '+xml
        if i in test_id:
            test_f.write(str)
            test_f.write('\n')
            continue
        train_f.write(str)
        train_f.write('\n')
    print('train:{}'.format(xml_num-test_num))
    print('test:{}'.format(test_num))
    train_f.close()
    test_f.close()
```

```
img_dir='**/train/IMAGES'
xml_dir='**/train/ANNOTATIONS'
train_txt='**/train.txt'
test_txt='**/test.txt'
#选一个运行即可
txt_get_all(img_dir,xml_dir,train_txt)
```

在datasets/oc目录下新建一个label.txt:
```
crazing
inclusion
pitted_surface
scratches
patches
rolled-in_scale
```

然后只需要配置voc.yml文件就可以使用ppdet内置的vocdataset了：
```
metric: VOC
map_type: 11point
num_classes: 6

TrainDataset:
  !VOCDataSet
    dataset_dir: /home/aistudio/work/data
    anno_path: train.txt
    label_list: label.txt
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

EvalDataset:
  !VOCDataSet
    dataset_dir: /home/aistudio/work/data
    anno_path: test.txt
    label_list: label.txt
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

TestDataset:
  !ImageFolder
    anno_path: dataset/voc/label.txt
```

然后关于reader的数据增强策略的话可以根据自己的需求更改，因为首先是抱着先实验的目的，所以暂时没有更改。

这样我们的数据管道就构建完成了

# 三.训练模型

以下训练模型以ppyoloe-plus-s为例

有兴趣可以看一下项目里的具体配置

下面列出ppyoloe_plus_crn_s_80e_coco.yml：

```
_BASE_: [
  '../datasets/voc.yml',
  '../runtime.yml',
  './_base_/optimizer_80e.yml',
  './_base_/ppyoloe_plus_crn.yml',
  './_base_/ppyoloe_plus_reader.yml',
]

log_iter: 100
snapshot_epoch: 5
weights: weights/best_model

depth_mult: 0.33
width_mult: 0.50


pretrain_weights: https://paddledet.bj.bcebos.com/models/ppyoloe_plus_crn_l_80e_coco.pdparams

```
voc.yml：voc数据集的说明
optimizer_300e:关于优化器的配置
ppyoloe_plus_crn：关于网络结构的配置
ppyoloe_plus_reader：关于数据管道的配置

按照自己的需求修改好配置文件之后就开始训练了：
```
python tools/train.py -c configs/ppyoloe/ppyoloe_plus_crn_s_80e_coco.yml
```
如果需要边训练边评估：
```
python tools/train.py -c configs/ppyoloe/ppyoloe_plus_crn_s_80e_coco.yml --eval
```




# 四.预测保存结果
训练好模型之后，就可以进行模型推理了。可以在ppyoloe_plus_reader配置文件下修改模型推理的相应的数据管道，比如修改推理图片的大小。当然在这里也需要将我们的检测结果保存到csv文件下，因此对infer的代码添加了一些代码(因为懒，当时报名的时候只剩下10月最后两三天了，代码可能写的有点粗糙，然后后面一直关于优化训练过程，忘记了这里)，并不是像大佬们一样写了一个json转换到csv文件的程序.

对trainer.py的Trainer.predict的修改如下(代码太长，所以只亮出了修改的部分)：
```
 if visualize:
            import csv
            if os.path.exists('submission.csv'):
                os.remove('submission.csv')

            f = open('submission.csv', 'w', encoding='UTF8', newline='')
            writer = csv.writer(f)
            writer.writerow(['image_id', 'bbox', 'category_id', 'confidence'])
            for outs in results:
                batch_res = get_infer_results(outs, clsid2catid,writer)
                bbox_num = outs['bbox_num']
```

对coco_utils.py的get_infer_results函数修改如下
```
def get_infer_results(outs, catid, writer,bias=0):
    
    if outs is None or len(outs) == 0:
        raise ValueError(
            'The number of valid detection result if zero. Please use reasonable model and check input data.'
        )

    im_id = outs['im_id']

    infer_res = {}
    if 'bbox' in outs:
        if len(outs['bbox']) > 0 and len(outs['bbox'][0]) > 6:
            infer_res['bbox'] = get_det_poly_res(
                outs['bbox'], outs['bbox_num'], im_id, catid, bias=bias)
        else:
            infer_res['bbox'] = get_det_res(
                outs['bbox'], outs['bbox_num'], im_id, catid, writer,bias=bias)
```

然后对json_result.py的get_det_res函数修改如下：
```
def get_det_res(bboxes, bbox_nums, image_id, label_to_cat_id_map, writer,bias=0):
    det_res = []
    k = 0
    for i in range(len(bbox_nums)):
        cur_image_id = int(image_id[i][0])
        det_nums = bbox_nums[i]
        for j in range(det_nums):
            dt = bboxes[k]
            k = k + 1
            num_id, score, xmin, ymin, xmax, ymax = dt.tolist()
            if xmin<0:
                xmin=0
            if ymin<0:
                ymin=0

            w_bbox='[{},{},{},{}]'.format(xmin,ymin,xmax,ymax)
            if int(num_id) < 0:
                continue
            category_id = label_to_cat_id_map[int(num_id)]
            info = [str(cur_image_id), w_bbox, str(category_id),str(score)]
            writer.writerow(info)
            w = xmax - xmin + bias
            h = ymax - ymin + bias
            bbox = [xmin, ymin, w, h]
            dt_res = {
                'image_id': cur_image_id,
                'category_id': category_id,
                'bbox': bbox,
                'score': score
            }
            det_res.append(dt_res)
    return det_res
```

然后还需要记得把配置文件ppyoloe_plus_crn_s_80e_coco.yml的weights改成自己的模型。

这样我们就可以预测了：
```
python tools/infer.py -c configs/ppyoloe/ppyoloe_plus_crn_s_80e_coco.yml
```
然后提交我们的csv文件即可。

# 写在最后
以前虽然说对飞浆有所了解，但是只有自己用起来才发现是真香。我们的训练集只有1400张图片，感觉有点少，于是就写了一个数据增强扩充数据的程序，但是还在开发中，后续可能再更新进来。最终用picodet取得了43的score，权重的话就在项目里面，但是有可能以前的某些权重我没有及时处理被覆盖掉了。训练日志也并没有保存下来。

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 